In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install polars "srai[all]" h3ronpy omegaconf wandb pytorch

^C
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


# Training GeoVex model


In [ ]:
import os
import sys
from pathlib import Path

# add the
ROOT = Path(os.getcwd())
# while not (ROOT / ".git").exists():
ROOT = ROOT

sys.path.append(str(ROOT))

from src.config import CargoBikeConfig, load_config
from src.osm_tags import build_tag_filter


import polars as pl
from h3ronpy.polars import grid_disk
from srai.h3 import ring_buffer_h3_indexes
from srai.neighbourhoods.h3_neighbourhood import H3Neighbourhood
from srai.embedders.geovex.model import GeoVexModel
from srai.embedders.geovex.dataset import HexagonalDataset

/home/shadeform/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


## Load Config


In [ ]:
config = load_config(ROOT / "config" / "paper.yaml")

target_tags = build_tag_filter(config)
target_tag_list = [f"{t}_{st}" for t, sts in target_tags.items() for st in sts]

## Load the Tag Data


In [ ]:
count_df_file = ROOT / "data" / "geovex" / "count.parquet"

if not count_df_file.exists():
    count_df_file.parent.mkdir(parents=True, exist_ok=True)

    count_df = pl.concat(
        pl.scan_parquet(city.count_file).with_columns(pl.lit(city.name).alias("city"))
        for city in config.Cities
    ).collect()

    count_df.write_parquet(
        count_df_file,
        compression="snappy",
    )
else:
    count_df = pl.read_parquet(count_df_file)

count_df.head()

amenity_bar,amenity_biergarten,amenity_cafe,amenity_fast_food,amenity_food_court,amenity_ice_cream,amenity_pub,amenity_restaurant,amenity_college,amenity_driving_school,amenity_kindergarten,amenity_language_school,amenity_library,amenity_toy_library,amenity_music_school,amenity_school,amenity_university,amenity_bicycle_parking,amenity_bicycle_repair_station,amenity_bicycle_rental,amenity_boat_rental,amenity_boat_sharing,amenity_bus_station,amenity_car_rental,amenity_car_sharing,amenity_car_wash,amenity_vehicle_inspection,amenity_charging_station,amenity_ferry_terminal,amenity_fuel,amenity_grit_bin,amenity_motorcycle_parking,amenity_parking,amenity_parking_entrance,amenity_parking_space,amenity_taxi,amenity_atm,…,sport_zurkhaneh_sport,water_river,water_oxbow,water_canal,water_ditch,water_lock,water_fish_pass,water_lake,water_reservoir,water_pond,water_basin,water_lagoon,water_stream_pool,water_reflecting_pool,water_moat,water_wastewater,waterway_river,waterway_riverbank,waterway_stream,waterway_tidal_channel,waterway_canal,waterway_drain,waterway_ditch,waterway_pressurised,waterway_fairway,waterway_dock,waterway_boatyard,waterway_dam,waterway_weir,waterway_waterfall,waterway_lock_gate,waterway_soakhole,waterway_turning_point,waterway_water_point,waterway_fuel,region_id,city
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""892a3029a33fff…","""Boston, USA"""
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""892a3065c8ffff…","""Boston, USA"""
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""892a302b077fff…","""Boston, USA"""
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""892a3067227fff…","""Boston, USA"""
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""892a339146ffff…","""Boston, USA"""


### Clean the Dataset


#### Drop Hexagons with No Building / Highway

We do this b.c. we are only focused on hexagons that could have deliveries


In [ ]:
keep_h3s = ring_buffer_h3_indexes(
    count_df.filter(
        (pl.sum_horizontal(pl.col("^building_.*$")) > 0)
        | (pl.sum_horizontal(pl.col("^highway_.*$")) > 0),
    )["region_id"],
    config.GeoVex.radius,
)

count_df = count_df.filter(pl.col("region_id").is_in(keep_h3s))

#### Drop Columns that Sum to Zero


In [ ]:
# drop columns that sum to 0
count_df = count_df.drop(
    count_df.select([pl.col(target_tag_list).sum() <= 0])
    .transpose(
        include_header=True,
        column_names=["drop"],
    )
    .filter(pl.col("drop"))["column"]
)

target_tag_list = sorted(set(count_df.columns).intersection(set(target_tag_list)))

In [ ]:
len(target_tag_list)

681

In [ ]:
# with open(ROOT / "data" / "geovex" / "target_tags.txt", "w") as f:
#     f.write("\n".join(target_tag_list))

## Create the Dataset


### Make a Training and Validation Set

Use 80/20 from each city


In [ ]:
training_h3s = (
    count_df.select(["city", "region_id"])
    .group_by("city")
    .agg(pl.col("region_id").sample(pl.count() * 0.9))
    .explode("region_id")
)

In [ ]:
test_h3s = count_df.select(["city", "region_id"]).filter(
    ~pl.col("region_id").is_in(training_h3s["region_id"])
)

In [ ]:
from torch.utils.data import DataLoader
from src.srai_patches.dataset import HexagonalDatasetPatch

data_df = (
    count_df.select(
        target_tag_list
        + [
            "region_id",
        ]
    )
    .to_pandas()
    .set_index("region_id")
)

train_dataset = HexagonalDatasetPatch(
    target_indexes=training_h3s["region_id"].to_list(),
    data=data_df,
    neighbourhood=H3Neighbourhood(),
    neighbor_k_ring=config.GeoVex.radius,
)

test_dataset = HexagonalDatasetPatch(
    target_indexes=test_h3s["region_id"].to_list(),
    data=data_df,
    neighbourhood=H3Neighbourhood(),
    neighbor_k_ring=config.GeoVex.radius,
)

  0%|          | 0/35964 [00:00<?, ?it/s]

100%|██████████| 8994/8994 [00:04<00:00, 2008.14it/s]


In [ ]:
len(
    set(train_dataset.get_valid_cells()).intersection(
        set(test_dataset.get_valid_cells())
    )
)

0

In [ ]:
num_cpus = os.cpu_count()

train_dataloader = DataLoader(
    train_dataset,
    batch_size=512,
    shuffle=True,
    num_workers=10,
    pin_memory=True,
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=512,
    shuffle=False,
    num_workers=4,
    persistent_workers=True,
    pin_memory=True,
)

## Train the Model


In [ ]:
model = GeoVexModel(
    k_dim=data_df.shape[1],
    radius=config.GeoVex.radius,
    conv_layers=3,
    emb_size=50,
    learning_rate=0.00025,
)

In [ ]:
import os
import torch
import wandb
from pytorch_lightning.loggers.wandb import WandbLogger
import pytorch_lightning as pl_lightning
from pytorch_lightning.callbacks import LambdaCallback, ModelCheckpoint
from pytorch_lightning import seed_everything

seed_everything(42)

# this is required for some reason to get the data to save correctly
wandb.finish()

epochs = 100

os.environ["WANDB_NOTEBOOK_NAME"] = "cluster-word2vec.ipynb"

logger = WandbLogger(
    project="GeoVex",
    log_model="all",
)

checkpoint_callback = ModelCheckpoint(
    monitor="train_loss",
    mode="min",
)

trainer = pl_lightning.Trainer(
    accelerator="cuda" if torch.cuda.is_available() else "cpu",
    max_epochs=epochs,
    logger=logger,
    callbacks=[checkpoint_callback],
    log_every_n_steps=20,
)

Seed set to 42
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### Train


In [ ]:
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

logger.experiment.finish()

run = logger.experiment


You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find cluster-word2vec.ipynb.
wandb: Currently logged in as: _max_ (green-last-mile). Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 4.4 M 
1 | decoder | Sequential | 3.2 M 
2 | _loss   | GeoVeXLoss | 0     
---------------------------------------
7.6 M     Trainable params
0         Non-trainable params
7.6 M     Total params
30.370    Total estimated model params size (MB)


Epoch 31:   0%|          | 0/24 [00:00<?, ?it/s, v_num=iwy3]               

/home/shadeform/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train_loss_epoch,█▅▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▇▆▅▄▁▃▃▃▃▃▁▃▃▃▃▃▁▃▃▃▃▃▁▃▃▃▃▃▁▃▃▃▂▂▁▂
trainer/global_step,▁▁▁▁▁▂▂▂▂▂▃▂▂▂▄▂▂▃▃▄▃▃▃▃▅▃▃▃▄▆▄▄▄▇▄▄▄▄█▅
validation_loss_epoch,█▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss_step,█▆▃▃▃▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁
epoch,30
train_loss_epoch,1204.00598
train_loss_step,1187.60461
trainer/global_step,743
validation_loss_epoch,709.26624


NameError: name 'train_df' is not defined

## Create the Embeddings


In [20]:
import pandas as pd
import numpy as np
from srai.constants import REGIONS_INDEX

data_df = (
    count_df.select(
        target_tag_list
        + [
            "region_id",
        ]
    )
    .to_pandas()
    .set_index("region_id")
)

dataset = HexagonalDataset(
    # target_indexes=training_h3s["region_id"].to_list(),
    data=data_df,
    neighbourhood=H3Neighbourhood(),
    neighbor_k_ring=config.GeoVex.radius,
)

dataloader = DataLoader(dataset, batch_size=1024, shuffle=False, num_workers=10)


embeddings = [
    model.cuda().encoder(batch.cuda()).cpu().detach().numpy() for batch in dataloader  # type: ignore
]

df = pd.DataFrame(np.concatenate(embeddings), index=dataset.get_valid_cells())
df.index.name = REGIONS_INDEX

df.to_parquet(Path(run.dir) / "embeddings.parquet", compression="snappy")

100%|██████████| 44958/44958 [00:23<00:00, 1917.49it/s]


In [21]:
(Path(run.dir) / "embeddings.parquet").absolute()

PosixPath('/home/shadeform/geovex/wandb/run-20240105_030315-eikqiwy3/files/embeddings.parquet')

In [22]:
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
region_id,,,,,,,,,,,,,,,,,,,,,
892a339a5afffff,3.008468,-6.218636,3.167253,-5.345560,3.089679,3.361192,2.645655,2.246335,0.023625,-1.522305,...,1.425137,4.334805,-4.044781,5.130377,2.705628,2.517802,-2.352568,1.447540,1.181890,-1.838629
892a3066a3bffff,2.900576,-4.953621,3.015917,-4.052948,2.526266,3.191200,2.278576,2.370381,-0.537876,-2.665453,...,0.971319,3.058970,-2.264176,4.277760,3.836319,2.361648,-2.357554,1.643557,1.242620,-1.947126
892a3066e17ffff,2.991765,-2.458879,1.748562,-2.870414,2.229912,4.270313,1.785876,4.174863,0.445850,-2.045090,...,-0.635486,3.441358,-0.276245,5.267743,1.940237,2.692774,-4.197291,2.992770,1.639096,-2.677036
892a3066b3bffff,2.417068,-5.508893,3.040622,-5.061748,2.756283,3.416097,2.153312,2.535958,-1.060595,-2.335514,...,0.369993,3.726042,-3.271048,4.367739,2.900891,2.333595,-2.681242,1.544899,1.432062,-2.209439
892a3066803ffff,2.756579,-5.839027,3.565192,-4.292718,1.611142,2.920071,1.804465,2.554294,0.637014,-2.318934,...,-0.468880,2.927581,-3.488756,3.423624,3.688035,3.346052,-2.700709,2.142959,0.231324,-2.851169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891fa44f4dbffff,3.660157,-5.766229,4.152099,-4.521164,3.523320,3.920768,2.894046,3.104259,-0.515101,-2.005774,...,-0.998065,4.350251,-3.790802,5.185569,4.034209,2.810549,-4.024291,1.962791,2.781991,-3.060663
891fa44060fffff,3.268006,-7.045927,3.990751,-5.940444,3.630116,3.836294,2.456876,3.495233,0.979446,-3.369663,...,0.737899,4.782680,-3.238692,4.672012,4.390110,3.258547,-3.119345,2.163912,0.260689,-2.559783
891fa441b9bffff,4.165100,-2.054721,1.722967,-0.608840,0.866216,3.124492,2.584760,2.581094,-0.235117,-3.362831,...,-0.948807,2.340863,0.015737,3.443399,4.500319,1.508626,-3.178720,2.718611,2.039006,-2.696587
